<a href="https://colab.research.google.com/github/BALAJIHARIDASAN/Natural-Language-Processing/blob/main/Natural_language_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import re
import string
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict
from collections import Counter

import keras
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM,
                          Embedding,
                          BatchNormalization,
                          Dense,
                          TimeDistributed,
                          Dropout,
                          Bidirectional,
                          Flatten,
                          GlobalMaxPool1D)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    accuracy_score
)




In [63]:
# steps in natural language processing

# load the dataset
# text preprocessing - 1
# text preprocessing -2
#

In [64]:
df_train = pd.read_csv('train.csv')

In [65]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [66]:
df_train = df_train[['text','target']]

In [67]:
df_train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [68]:
df_test = pd.read_csv('test.csv')

In [69]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [70]:
df_test = df_test[['text']]

In [71]:
df_test.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [72]:
df_train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [73]:
# loading the dataset
df_train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [74]:
df_train.columns

Index(['text', 'target'], dtype='object')

In [75]:
df_train.shape

(7613, 2)

In [76]:
df_train.columns

Index(['text', 'target'], dtype='object')

In [77]:
df_train.target.value_counts() # 1- disaster/ 0 - not disaster

0    4342
1    3271
Name: target, dtype: int64

In [78]:
df_train.text[0],df_train.target[0]

('Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all', 1)

In [79]:
# Text preprocessing

In [80]:
import re
import string
# Special thanks to https://www.kaggle.com/tanulsingh077 for this function
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [81]:
df_train['text_clean'] = df_train['text'].apply(clean_text)
df_train.head()

,text,target,text_clean
0,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...
4,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [82]:
stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

In [84]:
df_train['text_clean'] = df_train['text_clean'].apply(stemm_text)
df_train.head()

,text,target,text_clean
0,Our Deeds are the Reason of this #earthquake M...,1,our deed are the reason of this earthquak may ...
1,Forest fire near La Ronge Sask. Canada,1,forest fire near la rong sask canada
2,All residents asked to 'shelter in place' are ...,1,all resid ask to shelter in place are be notif...
3,"13,000 people receive #wildfires evacuation or...",1,peopl receiv wildfir evacu order in california
4,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from rubi alaska as s...


In [97]:
from nltk.corpus import stopwords
def preprocess_data(text):
    # Clean puntuation, urls, and so on
    text = clean_text(text)
    # Remove stopwords
    text = ' '.join(word for word in text.split(' '))
    # Stemm all the words in the sentence
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))

    return text

In [98]:
df_train['text_clean'] = df_train['text_clean'].apply(preprocess_data)
df_train.head()

,text,target,text_clean
0,Our Deeds are the Reason of this #earthquake M...,1,our deed are the reason of this earthquak may ...
1,Forest fire near La Ronge Sask. Canada,1,forest fire near la rong sask canada
2,All residents asked to 'shelter in place' are ...,1,all resid ask to shelter in place are be notif...
3,"13,000 people receive #wildfires evacuation or...",1,peopl receiv wildfir evacu order in california
4,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from rubi alaska as s...


In [100]:
# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
x = df_train['text_clean']
y = df_train['target']

print(len(x), len(y))

7613 7613


In [101]:
# Split into train and test sets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

5709 5709
1904 1904


In [102]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(x_train)

CountVectorizer()

In [103]:
# Use the trained to create a document-term matrix from train and test sets
x_train_dtm = vect.transform(x_train)
x_test_dtm = vect.transform(x_test)

In [104]:
vect_tunned = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.1, max_df=0.7, max_features=100)

In [105]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(x_train_dtm)
x_train_tfidf = tfidf_transformer.transform(x_train_dtm)

x_train_tfidf

<5709x11314 sparse matrix of type '<class 'numpy.float64'>'
	with 70660 stored elements in Compressed Sparse Row format>

In [107]:
texts = df_train['text_clean']
target = df_train['target']

In [108]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(texts)

vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

13750

In [109]:
tf.keras.preprocessing.sequence.pad_sequences(
    sequences, maxlen=None, dtype='int32', padding='pre',
    truncating='pre', value=0.0
)

array([[1],
       [2]], dtype=int32)

In [112]:
nltk.download('punkt')

def embed(corpus):
    return word_tokenizer.texts_to_sequences(corpus)

longest_train = max(texts, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(longest_train))

train_padded_sentences = pad_sequences(
    embed(texts),
    length_long_sentence,
    padding='post'
)

train_padded_sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


array([[ 137, 3746,   23, ...,    0,    0,    0],
       [ 217,   26,  178, ...,    0,    0,    0],
       [  40, 1445,  600, ...,    0,    0,    0],
       ...,
       [ 278,    5,  545, ...,    0,    0,    0],
       [  87,  243,   39, ...,    0,    0,    0],
       [   1,  253,   52, ...,    0,    0,    0]], dtype=int32)

In [113]:
import plotly.figure_factory as ff

x_axes = ['Ham', 'Spam']
y_axes =  ['Spam', 'Ham']

def conf_matrix(z, x=x_axes, y=y_axes):

    z = np.flip(z, 0)

    # change each element of z to type string for annotations
    z_text = [[str(y) for y in x] for x in z]

    # set up figure
    fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

    # add title
    fig.update_layout(title_text='<b>Confusion matrix</b>',
                      xaxis = dict(title='Predicted value'),
                      yaxis = dict(title='Real value')
                     )

    # add colorbar
    fig['data'][0]['showscale'] = True

    return fig

In [114]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# Train the model
nb.fit(x_train_dtm, y_train)

MultinomialNB()

In [115]:
# Make class anf probability predictions
y_pred_class = nb.predict(x_test_dtm)
y_pred_prob = nb.predict_proba(x_test_dtm)[:, 1]

In [116]:
# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

conf_matrix(metrics.confusion_matrix(y_test, y_pred_class))

0.8025210084033614


In [117]:
# Calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.8542519980653519

In [118]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()),
                 ('tfid', TfidfTransformer()),
                 ('model', MultinomialNB())])

In [119]:
# Fit the pipeline with the data
pipe.fit(x_train, y_train)

y_pred_class = pipe.predict(x_test)

print(metrics.accuracy_score(y_test, y_pred_class))

conf_matrix(metrics.confusion_matrix(y_test, y_pred_class))

0.8035714285714286


In [120]:
import xgboost as xgb

pipe = Pipeline([
    ('bow', CountVectorizer()),
    ('tfid', TfidfTransformer()),
    ('model', xgb.XGBClassifier(
        learning_rate=0.1,
        max_depth=7,
        n_estimators=80,
        use_label_encoder=False,
        eval_metric='auc',
        # colsample_bytree=0.8,
        # subsample=0.7,
        # min_child_weight=5,
    ))
])

In [121]:
# Fit the pipeline with the data
pipe.fit(x_train, y_train)

y_pred_class = pipe.predict(x_test)
y_pred_train = pipe.predict(x_train)

print('Train: {}'.format(metrics.accuracy_score(y_train, y_pred_train)))
print('Test: {}'.format(metrics.accuracy_score(y_test, y_pred_class)))

conf_matrix(metrics.confusion_matrix(y_test, y_pred_class))

Train: 0.8332457523208968
Test: 0.7589285714285714


In [142]:
xtrain, xvalid, ytrain, yvalid = train_test_split(df_train.text_clean.values, df_train.target.values,
                                                  stratify=df_train.target.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

In [147]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [148]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [157]:

# A simpleRNN without any pretrained embeddings and one dense layer
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                  300,
                  input_length=max_len))
model.add(SimpleRNN(10))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1500, 300)         4125000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 10)                3110      
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 4128121 (15.75 MB)
Trainable params: 4128121 (15.75 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [158]:
model.fit(xtrain_pad, ytrain, epochs=1, batch_size=32) #Multiplying

191/191 [==============================] - 91s 469ms/step - loss: 0.5921 - accuracy: 0.6852


In [169]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc


In [170]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

48/48 [==============================] - 21s 437ms/step
Auc: 0.73%


In [162]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/content/glove.6B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400000it [00:32, 12429.69it/s]

Found 400000 word vectors.


In [163]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 13749/13749 [00:00<00:00, 272831.07it/s]


In [172]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                  300,
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False))

model.add(LSTM(10, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 1500, 300)         4125000   
                                                                 
 lstm_5 (LSTM)               (None, 10)                12440     
                                                                 
 dense_13 (Dense)            (None, 1)                 11        
                                                                 
Total params: 4137451 (15.78 MB)
Trainable params: 12451 (48.64 KB)
Non-trainable params: 4125000 (15.74 MB)
_________________________________________________________________


In [173]:
model.fit(xtrain_pad, ytrain, epochs=1,batch_size = 32)

191/191 [==============================] - 284s 1s/step - loss: 0.5860 - accuracy: 0.6972


In [174]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

48/48 [==============================] - 9s 179ms/step
Auc: 0.84%


In [175]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                300,
                weights=[embedding_matrix],
                input_length=max_len,
                trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(10))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 1500, 300)         4125000   
                                                                 
 spatial_dropout1d (Spatial  (None, 1500, 300)         0         
 Dropout1D)                                                      
                                                                 
 gru (GRU)                   (None, 10)                9360      
                                                                 
 dense_14 (Dense)            (None, 1)                 11        
                                                                 
Total params: 4134371 (15.77 MB)
Trainable params: 9371 (36.61 KB)
Non-trainable params: 4125000 (15.74 MB)
_________________________________________________________________


In [177]:
model.fit(xtrain_pad, ytrain, epochs=1, batch_size=64)

96/96 [==============================] - 55s 573ms/step - loss: 0.5217 - accuracy: 0.7514


In [178]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

48/48 [==============================] - 7s 139ms/step
Auc: 0.84%


In [181]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                  300,
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False))
model.add(Bidirectional(LSTM(3, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])


model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 1500, 300)         4125000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 6)                 7296      
 onal)                                                           
                                                                 
 dense_16 (Dense)            (None, 1)                 7         
                                                                 
Total params: 4132303 (15.76 MB)
Trainable params: 7303 (28.53 KB)
Non-trainable params: 4125000 (15.74 MB)
_________________________________________________________________


In [182]:
model.fit(xtrain_pad, ytrain, epochs=1, batch_size=64)

96/96 [==============================] - 367s 4s/step - loss: 0.6523 - accuracy: 0.6225


In [183]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

48/48 [==============================] - 13s 269ms/step
Auc: 0.79%
